In [6]:
import calendar
import gc
import os
import sys

import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import scipy
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from implicit.als import AlternatingLeastSquares
import zipfile

In [7]:
os.getcwd()

'/home/mle-user/mle_projects/mle-pr-final/notebooks'

In [9]:
os.listdir('/home/mle-user/mle_projects/mle-pr-final/data')

['archive.zip']

In [10]:
arch_path = '/home/mle-user/mle_projects/mle-pr-final/data'
arch_name ='archive.zip'
dest = '/home/mle-user/mle_projects/mle-pr-final/data'

In [11]:
zip_path = os.path.join(arch_path, arch_name)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
     zip_ref.extractall(dest)

In [12]:
data_path = '/home/mle-user/mle_projects/mle-pr-final/data'
os.listdir(data_path)

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']